# AWS S3 Tables MCP Server on AgentCore Runtime

This notebook demonstrates how to test and deploy the AWS S3 Tables MCP server to Amazon Bedrock AgentCore Runtime.

## Prerequisites Setup

**Step 1:** Clone the AWS MCP repository
```bash
git clone https://github.com/awslabs/mcp.git
```

**Step 2:** Copy the AWS S3 Tables MCP server to your project root
```bash
cp -r ./mcp/src/s3-tables-mcp-server ./
```

**Step 3:** Set up your environment variables in `.env` file:
```
COGNITO_POOL_ID=your_pool_id
COGNITO_REGION=us-east-1
COGNITO_USERNAME=admin
COGNITO_CLIENT_SECRET=your_client_secret
COGNITO_PASSWORD=your_password
AWS_PROFILE=default
```

**Step 4:** Follow the instructions below to complete the deployment

## AWS S3 Tables MCP Server Overview

The AWS S3 Tables MCP server provides comprehensive S3-based table storage management capabilities through 16 specialized tools:

### Table Bucket Management Tools
- **`list_table_buckets`**: List all S3 table buckets in your AWS account
- **`create_table_bucket`**: Create new S3 Table Buckets to organize tabular data at scale
- **`get_bucket_metadata_config`**: Get metadata table configuration for regular S3 buckets

### Namespace Management Tools
- **`list_namespaces`**: List all namespaces across all S3 table buckets
- **`create_namespace`**: Create namespaces within table buckets for logical data separation

### Table Management Tools
- **`list_tables`**: List all S3 tables across all table buckets and namespaces
- **`create_table`**: Create individual tables within namespaces with flexible schema definition
- **`rename_table`**: Rename tables or move them to different namespaces
- **`get_table_metadata_location`**: Get the S3 URI location of table metadata
- **`update_table_metadata_location`**: Update the metadata location for S3 tables

### Maintenance and Monitoring Tools
- **`get_table_maintenance_config`**: Retrieve maintenance settings for tables
- **`get_maintenance_job_status`**: Get the status of maintenance jobs for tables

### Data Operations Tools
- **`query_database`**: Execute read-only SQL queries against S3 Tables using PyIceberg/Daft
- **`import_csv_to_table`**: Create S3 Tables from CSV files uploaded to S3
- **`import_parquet_to_table`**: Create S3 Tables from Parquet files uploaded to S3
- **`append_rows_to_table`**: Append new data rows to existing S3 tables

### Key Features
- **Read-Only Mode**: Optional security mode that restricts all operations to read-only
- **SQL Query Support**: Execute read-only SQL queries with insert-only write operations
- **Automatic Schema Inference**: Create tables from CSV/Parquet files with inferred schemas
- **Metadata Discovery**: Access comprehensive bucket metadata through S3 Metadata Tables
- **Flexible Data Modeling**: Support for Apache Iceberg format with partitioning

All tools work with S3-based table storage and require appropriate AWS S3 Tables permissions.

## Prerequisites

Before running this notebook, ensure you have:

### System Requirements
- Python 3.10 or higher
- AWS CLI configured with valid credentials
- Docker installed (for containerization)

### AWS Permissions
Your AWS credentials must have permissions for:
- AWS S3 Tables (full access recommended)
- Amazon S3 (for data storage and retrieval)
- Amazon Bedrock AgentCore
- Amazon ECR (for container registry)
- Amazon Cognito (for authentication)
- IAM (for role creation)
- AWS Systems Manager Parameter Store
- AWS Secrets Manager

### Project Structure
- `s3-tables-mcp-server/` - The MCP server implementation
- `requirements.txt` - Python dependencies
- `utils.py` - Helper functions for Cognito and IAM setup

## 1. Install Dependencies

Install all required Python packages using uv (recommended) or pip:

In [12]:
!uv pip install -r s3-table-requirements.txt

Using Python 3.13.5 environment at: mcp_venv
Resolved 78 packages in 514ms                                        
⠙ Preparing packages... (0/15)                                                  
⠙ Preparing packages... (0/15)-------------     0 B/30.85 KiB           
⠙ Preparing packages... (0/15)-------------     0 B/30.85 KiB           
pycodestyle          ------------------------------     0 B/30.85 KiB
⠙ Preparing packages... (0/15)-------------     0 B/44.73 KiB           
pycodestyle          ------------------------------     0 B/30.85 KiB
⠙ Preparing packages... (0/15)------------- 14.91 KiB/44.73 KiB         
pycodestyle          ------------------------------     0 B/30.85 KiB
⠙ Preparing packages... (0/15)------------- 14.91 KiB/44.73 KiB         
pycodestyle          ------------------------------     0 B/30.85 KiB
⠙ Preparing packages... (0/15)------------- 14.91 KiB/44.73 KiB         
pycodestyle          ------------------------------     0 B/30.85 KiB
autopep8        

## 2. Local Testing

Before deploying to AgentCore Runtime, let's test the AWS S3 Tables MCP server locally.

### 2.1 MCP Server Wrapper

The `mcp-server.py` file creates a FastMCP wrapper around the AWS S3 Tables MCP server for AgentCore deployment. The reason we have to do this is because AWS MCP servers are implemented for Stdio local run, we are extending it for remote MCP server using this wrapper:

- Imports all 16 tools from the original server implementation
- Configures the server for HTTP transport on port 8000
- Sets up proper instructions and dependencies
- Enables stateless HTTP mode for AgentCore compatibility

The server exposes these tools:
- Table bucket management: `list_table_buckets`, `create_table_bucket`, `get_bucket_metadata_config`
- Namespace management: `list_namespaces`, `create_namespace`
- Table management: `list_tables`, `create_table`, `rename_table`, `get_table_metadata_location`, `update_table_metadata_location`
- Maintenance: `get_table_maintenance_config`, `get_maintenance_job_status`
- Data operations: `query_database`, `import_csv_to_table`, `import_parquet_to_table`, `append_rows_to_table`

In [13]:
%%writefile mcp-server.py
#!/usr/bin/env python3
import os
import sys

# Add the path before any imports
sys.path.insert(0, os.path.abspath("./s3-tables-mcp-server"))

# Import the tool functions from the original server
from awslabs.s3_tables_mcp_server.server import (
    list_table_buckets, create_table_bucket, get_bucket_metadata_config,
    list_namespaces, create_namespace,
    list_tables, create_table, rename_table, get_table_metadata_location, update_table_metadata_location,
    get_table_maintenance_config, get_maintenance_job_status,
    query_database, import_csv_to_table, import_parquet_to_table, append_rows_to_table
)

# Create a new FastMCP instance with correct parameters
from mcp.server.fastmcp import FastMCP

mcp = FastMCP(
    'awslabs.s3-tables-mcp-server',
    host="0.0.0.0",
    stateless_http=True,
    instructions="""AWS S3 Tables MCP Server provides comprehensive tools for managing S3-based table storage through Amazon S3 Tables.

    This server enables you to:
    - Create and manage S3 Table Buckets for organizing tabular data at scale
    - Define namespaces within table buckets for logical data separation
    - Create, rename, and manage individual S3 tables with flexible schema definition
    - Execute read-only SQL queries against S3 Tables using PyIceberg/Daft engine
    - Import data from CSV and Parquet files stored in S3
    - Append new data rows to existing tables
    - Access comprehensive metadata and maintenance configurations
    - Discover bucket metadata through S3 Metadata Tables

    ## Available Tools:
    
    ### Table Bucket Management
    - list_table_buckets: List all S3 table buckets in your account
    - create_table_bucket: Create new table buckets for data organization
    - get_bucket_metadata_config: Get metadata configuration for S3 buckets
    
    ### Namespace Management
    - list_namespaces: List all namespaces across table buckets
    - create_namespace: Create logical groupings within table buckets
    
    ### Table Management
    - list_tables: List all tables across buckets and namespaces
    - create_table: Create tables with Apache Iceberg format and custom schemas
    - rename_table: Rename tables or move them between namespaces
    - get_table_metadata_location: Get S3 URI of table metadata
    - update_table_metadata_location: Update metadata location for tables
    
    ### Maintenance and Monitoring
    - get_table_maintenance_config: Retrieve table maintenance settings
    - get_maintenance_job_status: Monitor maintenance job status
    
    ### Data Operations
    - query_database: Execute SQL queries against S3 Tables (read-only with insert support)
    - import_csv_to_table: Create tables from CSV files with schema inference
    - import_parquet_to_table: Create tables from Parquet files with schema inference
    - append_rows_to_table: Add new data rows to existing tables

    The server supports both read-only and write operations, with optional security modes for production use.

    For more information about AWS S3 Tables, visit:
    https://aws.amazon.com/s3/features/tables/
    """,
    dependencies=[
        'pydantic',
        'loguru',
        'boto3',
        'pyiceberg',
        'daft-io',
    ],
)

# Register all the tools from the original server
mcp.tool(name='list_table_buckets')(list_table_buckets)
mcp.tool(name='create_table_bucket')(create_table_bucket)
mcp.tool(name='get_bucket_metadata_config')(get_bucket_metadata_config)
mcp.tool(name='list_namespaces')(list_namespaces)
mcp.tool(name='create_namespace')(create_namespace)
mcp.tool(name='list_tables')(list_tables)
mcp.tool(name='create_table')(create_table)
mcp.tool(name='rename_table')(rename_table)
mcp.tool(name='get_table_metadata_location')(get_table_metadata_location)
mcp.tool(name='update_table_metadata_location')(update_table_metadata_location)
mcp.tool(name='get_table_maintenance_config')(get_table_maintenance_config)
mcp.tool(name='get_maintenance_job_status')(get_maintenance_job_status)
mcp.tool(name='query_database')(query_database)
mcp.tool(name='import_csv_to_table')(import_csv_to_table)
mcp.tool(name='import_parquet_to_table')(import_parquet_to_table)
mcp.tool(name='append_rows_to_table')(append_rows_to_table)

if __name__ == "__main__":
    print("Starting MCP server in HTTP mode on http://0.0.0.0:8000")
    mcp.run(transport="streamable-http")

Overwriting mcp-server.py


### 2.2 Local Test Client

The `mcp-client.py` creates a simple test client that:

- Connects to the local MCP server at `http://0.0.0.0:8000/mcp`
- Lists all available tools
- Provides basic connectivity testing

This client helps verify that your MCP server is running correctly before deployment.

In [14]:
%%writefile mcp-client.py
#!/usr/bin/env python3
import asyncio
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def test_server():
    mcp_url = "http://0.0.0.0:8000/mcp"
    
    try:
        async with streamablehttp_client(mcp_url, {}, terminate_on_close=False) as (
            read_stream, write_stream, _
        ):
            async with ClientSession(read_stream, write_stream) as session:
                await session.initialize()
                
                tool_result = await session.list_tools()
                print(f"Found {len(tool_result.tools)} tools:")
                
                for tool in tool_result.tools:
                    print(f"  - {tool.name}")
                
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    asyncio.run(test_server())

Overwriting mcp-client.py


### 2.3 Local Testing Instructions

To test your AWS S3 Tables MCP server locally:

1. **Terminal 1**: Start the MCP server
   ```bash
   python mcp-server.py
   ```
   Expected output: `Starting MCP server in HTTP mode on http://0.0.0.0:8000`
   
2. **Terminal 2**: Run the test client
   ```bash
   python mcp-client.py
   ```
   Expected output: `Found 16 tools:` followed by the list of S3 Tables tools

**Note**: Local testing requires AWS credentials to be configured with access to AWS S3 Tables, as the tools make actual calls to AWS S3 Tables services.

## 3. Amazon Cognito Authentication Setup

AgentCore Runtime requires JWT-based authentication. We'll use Amazon Cognito to provide bearer tokens for accessing our deployed MCP server.

The `utils.py` file contains helper functions:
- `get_cognito_pool_info()`: Retrieves configuration from an existing Cognito User Pool
- `setup_cognito_user_pool()`: Creates a new User Pool if needed
- `create_agentcore_role()`: Creates the necessary IAM role with proper permissions

In [15]:
import os
from dotenv import load_dotenv
from utils import get_cognito_pool_info, create_agentcore_role

load_dotenv()

pool_id = os.getenv('COGNITO_POOL_ID', 'us-east-1_MYwiFq4Om')
region = os.getenv('COGNITO_REGION', 'us-east-1')
    
print(f"Get Cognito user pool info for pool id: {pool_id} in region: {region}")

print("Setting up Amazon Cognito user pool...")
cognito_config = get_cognito_pool_info(pool_id, region)
print("Cognito setup completed ✓")

Get Cognito user pool info for pool id: eu-central-1_PaVtjk8dt in region: eu-central-1
Setting up Amazon Cognito user pool...
Using client ID: 4rit5a00iqft9ak8sl5hb28sr
Making OAuth request to: https://mcp-registry-241533163649-mcp-gateway-registry.auth.eu-central-1.amazoncognito.com/oauth2/token
✅ Successfully obtained bearer token via OAuth client credentials
Pool id: eu-central-1_PaVtjk8dt
Discovery URL: https://cognito-idp.eu-central-1.amazonaws.com/eu-central-1_PaVtjk8dt/.well-known/openid-configuration
Client ID: 4rit5a00iqft9ak8sl5hb28sr
Bearer Token: eyJraWQiOiJYZ1wvblZQWmtZeEtLM3ZicHBPRVQ2cUVYZUZLdE12QkVcLzVyWjJGK3ZoZWs9IiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiI0cml0NWEwMGlxZnQ5YWs4c2w1aGIyOHNyIiwidG9rZW5fdXNlIjoiYWNjZXNzIiwic2NvcGUiOiJtY3AtcmVnaXN0cnlcL3JlYWQgbWNwLXJlZ2lzdHJ5XC93cml0ZSIsImF1dGhfdGltZSI6MTc1NTIxMjMzOCwiaXNzIjoiaHR0cHM6XC9cL2NvZ25pdG8taWRwLmV1LWNlbnRyYWwtMS5hbWF6b25hd3MuY29tXC9ldS1jZW50cmFsLTFfUGFWdGprOGR0IiwiZXhwIjoxNzU1MjE1OTM4LCJpYXQiOjE3NTUyMTIzMzgsInZlcnNpb24iOjIs

## 4. IAM Role Creation

Create an IAM execution role for the AgentCore Runtime with:

### Base Permissions
- Amazon Bedrock model invocation
- Amazon ECR image access
- CloudWatch logging
- X-Ray tracing
- AgentCore workload identity access

### Additional Permissions
- `AmazonS3FullAccess` - Required for S3 operations and data storage
- Custom S3 Tables policy - Required for all S3 Tables operations

The role is automatically configured with the proper trust policy for bedrock-agentcore.amazonaws.com service.

In [16]:
tool_name = "s3_tables_mcp_server"
additional_managed_policies = ['AmazonS3FullAccess']
print(f"Creating IAM role for {tool_name}...")
agentcore_iam_role = create_agentcore_role(
    agent_name=tool_name, 
    managed_policies=additional_managed_policies
)
print(f"IAM role created ✓")
print(f"Role ARN: {agentcore_iam_role['Role']['Arn']}")

Creating IAM role for s3_tables_mcp_server...
attaching inline role policy agentcore-s3_tables_mcp_server-role
Attaching 1 managed policies...
Attaching managed policy: AmazonS3FullAccess
✅ Successfully attached AmazonS3FullAccess
IAM role created ✓
Role ARN: arn:aws:iam::241533163649:role/agentcore-s3_tables_mcp_server-role


## 5. AgentCore Runtime Configuration

Configure the AgentCore Runtime deployment using the Bedrock AgentCore Starter Toolkit:

### Configuration Parameters
- **Entrypoint**: `mcp-server.py` (our FastMCP wrapper)
- **Execution Role**: The IAM role created above
- **Requirements**: `s3-table-requirements.txt` with all dependencies
- **Protocol**: MCP (Model Context Protocol)
- **Authentication**: Custom JWT authorizer with Cognito

### Auto-Generated Resources
- Dockerfile optimized for the MCP server
- Amazon ECR repository for container storage
- AgentCore Runtime configuration

The configuration validates that all required files exist before proceeding.

In [17]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp-server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp-server.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="s3-table-requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name=tool_name
)
print("Configuration completed ✓")

Entrypoint parsed: file=/Users/jingswu/Desktop/code-lab/aws-mcp-server-agentcore/mcp-server.py, bedrock_agentcore_name=mcp-server
Configuring BedrockAgentCore agent: s3_tables_mcp_server


Using AWS region: eu-central-1
All required files found ✓
Configuring AgentCore Runtime...


Generated Dockerfile: /Users/jingswu/Desktop/code-lab/aws-mcp-server-agentcore/Dockerfile
Generated .dockerignore: /Users/jingswu/Desktop/code-lab/aws-mcp-server-agentcore/.dockerignore
Setting 's3_tables_mcp_server' as default agent
Bedrock AgentCore configured: /Users/jingswu/Desktop/code-lab/aws-mcp-server-agentcore/.bedrock_agentcore.yaml


Configuration completed ✓


## 6. Deployment to AgentCore Runtime

Launch the MCP server to AgentCore Runtime. This process:

### Build and Deploy Steps
1. **Container Build**: Creates Docker image from the generated Dockerfile
2. **ECR Push**: Uploads the container to Amazon ECR
3. **Runtime Creation**: Deploys the AgentCore Runtime
4. **Service Registration**: Registers the MCP server endpoint

### Expected Outputs
- Agent ARN: Unique identifier for the deployed runtime
- Agent ID: Short identifier for management operations
- ECR URI: Container image location

**Note**: This process typically takes 5-10 minutes to complete.

In [18]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 's3_tables_mcp_server' to account 241533163649 (eu-central-1)
Starting CodeBuild ARM64 deployment for agent 's3_tables_mcp_server' to account 241533163649 (eu-central-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: s3_tables_mcp_server


Launching MCP server to AgentCore Runtime...
This may take several minutes...
Repository doesn't exist, creating new ECR repository: bedrock-agentcore-s3_tables_mcp_server


✅ ECR repository available: 241533163649.dkr.ecr.eu-central-1.amazonaws.com/bedrock-agentcore-s3_tables_mcp_server
Using execution role from config: arn:aws:iam::241533163649:role/agentcore-s3_tables_mcp_server-role
✅ Execution role validation passed: arn:aws:iam::241533163649:role/agentcore-s3_tables_mcp_server-role
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: s3_tables_mcp_server
Role name: AmazonBedrockAgentCoreSDKCodeBuild-eu-central-1-7bdf16fb13
Reusing existing CodeBuild execution role: arn:aws:iam::241533163649:role/AmazonBedrockAgentCoreSDKCodeBuild-eu-central-1-7bdf16fb13
Using .dockerignore with 44 patterns
Uploaded source to S3: s3_tables_mcp_server/20250814-225927.zip
Updated CodeBuild project: bedrock-agentcore-s3_tables_mcp_server-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 5.2s
🔄 PROVISIONING started (t

Launch completed ✓
Agent ARN: arn:aws:bedrock-agentcore:eu-central-1:241533163649:runtime/s3_tables_mcp_server-wxGgd74lvQ
Agent ID: s3_tables_mcp_server-wxGgd74lvQ


## 7. Runtime Status Monitoring

Monitor the AgentCore Runtime deployment status:

### Status States
- **CREATING**: Runtime is being deployed
- **READY**: Runtime is operational and ready to serve requests
- **CREATE_FAILED**: Deployment failed
- **UPDATE_FAILED**: Update operation failed
- **DELETE_FAILED**: Deletion operation failed

The monitoring loop checks status every 10 seconds until reaching a terminal state. Only proceed to testing when status is **READY**.

In [21]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")

Retrieved Bedrock AgentCore status for: s3_tables_mcp_server


Initial status: READY
✓ AgentCore Runtime is READY!


## 8. Configuration Storage

Store deployment configuration for remote access:

### AWS Systems Manager Parameter Store
- **Parameter**: `/s3_tables_mcp_server/runtime/agent_arn`
- **Value**: The Agent ARN from deployment
- **Purpose**: Runtime endpoint identification

### AWS Secrets Manager
- **Secret**: `s3_tables_mcp_server/cognito/credentials`
- **Content**: Complete Cognito configuration including bearer token
- **Purpose**: Authentication for remote client access

This configuration enables the remote client to authenticate and connect to the deployed MCP server.

In [22]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='s3_tables_mcp_server/cognito/credentials',
        Description='Cognito credentials for S3 Tables MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='s3_tables_mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/s3_tables_mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for S3 Tables MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

✓ Cognito credentials updated in Secrets Manager
✓ Agent ARN stored in Parameter Store

Configuration stored successfully!
Agent ARN: arn:aws:bedrock-agentcore:eu-central-1:241533163649:runtime/s3_tables_mcp_server-wxGgd74lvQ


In [23]:
%%writefile mcp_client_remote.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/s3_tables_mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='s3_tables_mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json",
        "Accept": "application/json, text/event-stream"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🗂️  {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} S3 Tables tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_client_remote.py


## 9. Remote Testing

Test the deployed MCP server using the remote client (`mcp_client_remote.py`):

### Remote Client Features
- **Credential Retrieval**: Automatically fetches Agent ARN and bearer token from AWS
- **HTTPS Connection**: Connects to the AgentCore Runtime endpoint
- **Tool Discovery**: Lists all available S3 Tables tools
- **Error Handling**: Provides detailed error messages for troubleshooting

### Expected Output
The client should display all 16 AWS S3 Tables tools with their descriptions and parameters:
- Table bucket management tools (3)
- Namespace management tools (2)
- Table management tools (5)
- Maintenance and monitoring tools (2)
- Data operations tools (4)

### Connection Details
- **Endpoint**: `https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations`
- **Authentication**: Bearer token from Cognito
- **Protocol**: MCP over HTTPS

In [24]:
print("Testing deployed MCP server...")
print("=" * 50)
!python mcp_client_remote.py

Testing deployed MCP server...
Using AWS region: eu-central-1
Retrieved Agent ARN: arn:aws:bedrock-agentcore:eu-central-1:241533163649:runtime/s3_tables_mcp_server-wxGgd74lvQ
✓ Retrieved bearer token from Secrets Manager

Connecting to: https://bedrock-agentcore.eu-central-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aeu-central-1%3A241533163649%3Aruntime%2Fs3_tables_mcp_server-wxGgd74lvQ/invocations?qualifier=DEFAULT

🔄 Initializing MCP session...

📋 Available MCP Tools:
🗂️  list_table_buckets
   Description: List all S3 table buckets for your AWS account.

Permissions:
You must have the s3tables:ListTableBuckets permission to use this operation.

   Parameters: ['region_name']

🗂️  create_table_bucket
   Description: Creates an S3 table bucket.

Permissions:
You must have the s3tables:CreateTableBucket permission to use this operation.

   Parameters: ['name', 'region_name']

🗂️  get_bucket_metadata_config
   Description: Get the metadata table configuration for a regular 

## 10. Resource Cleanup

Clean up all AWS resources created during this demonstration:

### Cleanup Operations
1. **Local Configuration**: Remove `.bedrock_agentcore.yaml`
2. **AgentCore Runtime**: Delete the deployed runtime
3. **ECR Repository**: Remove container images and repository
4. **IAM Role**: Detach policies and delete execution role
5. **Parameter Store**: Remove stored Agent ARN
6. **Secrets Manager**: Delete Cognito credentials

### Important Notes
- Cleanup is irreversible - ensure you no longer need the deployed resources
- Some resources may have dependencies that prevent immediate deletion
- Manual cleanup may be required if automated cleanup fails

**Warning**: This will permanently delete your deployed MCP server and all associated resources.

In [11]:
import boto3

print("🗑️  Starting cleanup process...")

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
iam_client = boto3.client('iam')
ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    print("Clean up agentcore-config file")
    config_file = ".bedrock_agentcore.yaml"
    if os.path.exists(config_file):
        os.remove(config_file)
        print(f"✅ Successfully removed {config_file}")

    print("Deleting AgentCore Runtime...")
    runtime_delete_response = agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=launch_result.agent_id,
    )
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_repo_name = launch_result.ecr_uri.split('/')[1]
    ecr_client.delete_repository(
        repositoryName=ecr_repo_name,
        force=True
    )
    print("✓ ECR repository deleted")

    print("Deleting IAM role policies...")
    policies = iam_client.list_role_policies(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        MaxItems=100
    )

    for policy_name in policies['PolicyNames']:
        iam_client.delete_role_policy(
            RoleName=agentcore_iam_role['Role']['RoleName'],
            PolicyName=policy_name
        )
    
    # List attached managed policies
    policies = iam_client.list_attached_role_policies(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        MaxItems=100
        )

    for policy in policies['AttachedPolicies']:
        policy_arn = policy['PolicyArn']
        iam_client.detach_role_policy(
            RoleName=agentcore_iam_role['Role']['RoleName'],
            PolicyArn=policy_arn
        )

    iam_client.delete_role(
        RoleName=agentcore_iam_role['Role']['RoleName']
    )
    print("✓ IAM role deleted")

    try:
        ssm_client.delete_parameter(Name='/s3_tables_mcp_server/runtime/agent_arn')
        print("✓ Parameter Store parameter deleted")
    except ssm_client.exceptions.ParameterNotFound:
        print("ℹ️  Parameter Store parameter not found")

    try:
        secrets_client.delete_secret(
            SecretId='s3_tables_mcp_server/cognito/credentials',
            ForceDeleteWithoutRecovery=True
        )
        print("✓ Secrets Manager secret deleted")
    except secrets_client.exceptions.ResourceNotFoundException:
        print("ℹ️  Secrets Manager secret not found")

    print("\n✅ Cleanup completed successfully!")
    
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

🗑️  Starting cleanup process...
Clean up agentcore-config file
✅ Successfully removed .bedrock_agentcore.yaml
Deleting AgentCore Runtime...
✓ AgentCore Runtime deletion initiated
Deleting ECR repository...
✓ ECR repository deleted
Deleting IAM role policies...
✓ IAM role deleted
✓ Parameter Store parameter deleted
✓ Secrets Manager secret deleted

✅ Cleanup completed successfully!
